# Computation of Expectation Values

This guide shows practical patterns to compute expectation values of the form

$⟨𝑂⟩_𝜓=⟨ψ∣O∣ψ⟩$

using ffsim, where
𝑂 can be a Hamiltonian or any other operator supported by `ffsim.linear_operator`.

We cover:

- expectation values from a state vector,

- expectation values for operators given as a FermionOperator.

### Prerequisites

We again use:

`norb`: number of spatial orbitals

`nelec = (n_alpha, n_beta)`: number of alpha, beta electrons

𝜓 is a state vector in the corresponding Fock basis sector

In ffsim, the dimension of the sector can be found via:

`ffsim.dim(norb, nelec)`.

In [59]:
import numpy as np

import ffsim

rng = np.random.default_rng(0)

norb = 3
nelec = (2, 1)
dim = ffsim.dim(norb, nelec)

# A random normalized complex state vector |psi>
psi = ffsim.random.random_state_vector(dim, seed=rng)
psi = psi / np.linalg.norm(psi)

print("Dimension:", dim, ",", "the normalized length:", np.linalg.norm(psi))

Dimension: 9 , the normalized length: 1.0


## Method 1: Use a LinearOperator

Most operators in ffsim can be converted to a `scipy.sparse.linalg.LinearOperator` using:

`linop = ffsim.linear_operator(op, norb=norb, nelec=nelec)`

Then compute

$⟨𝑂⟩_𝜓=⟨ψ∣(Oψ)⟩$

with `np.vdot`, which performs the conjugate dot product.

In [60]:
# Random but valid fermionic Hamiltonian
rng = np.random.default_rng(0)

H = ffsim.random.random_molecular_hamiltonian(
    norb=norb,
    seed=rng,
)

In [61]:
# Convert Hamiltonian to a LinearOperator
H_linop = ffsim.linear_operator(H, norb=norb, nelec=nelec)

# Expectation value
expval_linop = np.vdot(psi, H_linop @ psi).real

print("⟨ψ|H|ψ⟩ (LinearOperator):", expval_linop)

⟨ψ|H|ψ⟩ (LinearOperator): -34.11901910701321


### Notes

- `random_molecular_hamiltonian` generates a valid random one- and two-body Hamiltonian.
- LinearOperator avoids building dense matrices; it supports efficient matrix–vector products.
- For Hermitian operators, the expectation value is real up to numerical noise.
- FermionOperator is an explicit second-quantized representation; you can convert it to a LinearOperator the same way.

## Method 2: Start from a FermionOperator

If your operator is expressed as a `FermionOperator`, you can compute expectation values the same way:

Build a `FermionOperator` (e.g., from a Hamiltonian object), then

Convert it to a linear operator and compute $⟨ψ∣O∣ψ⟩$.

In [62]:
# Convert Hamiltonian to a FermionOperator
op = ffsim.fermion_operator(H)

# Convert FermionOperator to LinearOperator
op_linop = ffsim.linear_operator(op, norb=norb, nelec=nelec)

# Expectation value of FermionOperator Method
expval_fermion = np.vdot(psi, op_linop @ psi).real

print("⟨ψ|H|ψ⟩ (FermionOperator method):", expval_fermion)

# Check agreement
np.testing.assert_allclose(expval_fermion, expval_linop, rtol=1e-12, atol=1e-12)

⟨ψ|H|ψ⟩ (FermionOperator method): -34.119019107013216


## Example: Expectation values with `MolecularHamiltonian` (from PySCF)

In this example we build a molecular Hamiltonian for \(\mathrm{N_2}\) using PySCF,
convert it to an `ffsim.MolecularHamiltonian`, and compute the expectation value

$\langle H \rangle_\psi = \langle \psi | H | \psi \rangle$

in a fixed $(n_{\alpha}, n_{\beta})$ particle-number subspace.

We demonstrate two equivalent workflows:

1. `MolecularHamiltonian → LinearOperator`
2. `MolecularHamiltonian → FermionOperator → LinearOperator`

Both yield the same expectation value (up to numerical tolerance).


In [63]:
import pyscf
import pyscf.data.elements

# -----------------------------
# Build N2 molecule (minimal example)
# -----------------------------
mol = pyscf.gto.Mole()
mol.build(
    atom=[["N", (0, 0, 0)], ["N", (1.0, 0, 0)]],
    basis="sto-6g",
    symmetry="Dooh",
)

# Freeze core orbitals (chemically frozen core)
n_frozen = pyscf.data.elements.chemcore(mol)
active_space = range(n_frozen, mol.nao_nr())

# Run RHF
scf = pyscf.scf.RHF(mol).run()

# Convert to ffsim MolecularData in the active space
mol_data = ffsim.MolecularData.from_scf(scf, active_space=active_space)

norb, nelec = mol_data.norb, mol_data.nelec
H = mol_data.hamiltonian  # this is an ffsim MolecularHamiltonian

print(f"norb = {norb}")
print(f"nelec = {nelec}")

converged SCF energy = -108.464957764796
norb = 8
nelec = (5, 5)


In [64]:
dim = ffsim.dim(norb, nelec)

# random normalized state
rng = np.random.default_rng(12345)
psi_rand = ffsim.random.random_state_vector(dim, seed=rng)
psi_rand /= np.linalg.norm(psi_rand)

print("||psi_rand|| =", np.linalg.norm(psi_rand))

||psi_rand|| = 1.0000000000000002


In [65]:
# Build linear operator in the (norb, nelec) subspace
H_linop = ffsim.linear_operator(H, norb=norb, nelec=nelec)

# Expectation value of Hamiltonian for random state
expval_h_lin = np.vdot(psi_rand, H_linop @ psi_rand).real
print("⟨psi|H|psi⟩ =", expval_h_lin)

⟨psi|H|psi⟩ = -104.79898902331776


In [66]:
# Convert MolecularHamiltonian -> FermionOperator
op = ffsim.fermion_operator(H)

# FermionOperator -> LinearOperator
op_linop = ffsim.linear_operator(op, norb=norb, nelec=nelec)

# Expectation values again
expval_h_ferm = np.vdot(psi_rand, op_linop @ psi_rand).real
print("⟨psi|H|psi⟩ via FermionOperator =", expval_h_ferm)

# Check agreement
np.testing.assert_allclose(expval_h_lin, expval_h_ferm, rtol=1e-10, atol=1e-10)

⟨psi|H|psi⟩ via FermionOperator = -104.79898902331776


## Summary

Use `ffsim.linear_operator(op, norb, nelec)` and then compute np.vdot(psi, linop @ psi) for the `MolecularHamiltonian` since, it contains general two-body interactions and is not diagonal in the occupation-number basis.
`FermionOperator` offers a clear second-quantized representation and integrates seamlessly into the same workflow.